# GoldenSAML Quiz: Named Pipe

#### Dataset

- Windows Events: `sqlalchemy://GoldenSAML-WindowsEvents`
- MDE: `sqlalchemy://GoldenSAML-Microsoft365DefenderEvents`

The datasources will be automatically listed in the `FROM` clause of a `GET` command when pressing `tab` to auto-complete.

### Task A: Get Named Pipe (File)

1. Use a `GET` command to obtain named pipe with pattern `%##WID\tsql\query%` in `GoldenSAML-WindowsEvents`
2. Use raw string `r'%##WID\tsql\query%'` to avoid escaping the special characters in the pattern
3. Think about whether you should use `=` or `LIKE` as the operator in the `WHERE` clause
4. Check [OCSF file doc](https://schema.ocsf.io/1.3.0/objects/file) to see which attribute to use
5. Add a `DISP` command to display the object obtained

In [1]:
wid_named_pipe = GET file FROM sqlalchemy://GoldenSAML-WindowsEvents
                 WHERE name LIKE r'%##WID\tsql\query%'
                    
DISP wid_named_pipe

endpoint.hostname,endpoint.uid,name,type_id
adfs01.simulandlabs.com,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,\MICROSOFT##WID\tsql\query,6


### Task B: Find The Process Created The Pipe

**Prerequisite: Task A**

1. Use a `FIND` command to obtain processes that created the named pipe in Task A
2. Add a `DISP` command to display the processes obtained

In [2]:
reader = FIND process CREATED wid_named_pipe

DISP reader

endpoint.hostname,endpoint.uid,name,file.name,file.path,file.parent_folder,pid,uid
adfs01.simulandlabs.com,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,powershell.exe,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0,5692,9acedb82-ccd4-6107-22de-030000000600


### Task C: Connect to MDE Logs Associated With the Process

**Prerequisite: Task B**

1. Use a `GET` command to filter `event` from the MDE logs `GoldenSAML-Microsoft365DefenderEvents`

2. The events have `actor.process` that is the processes identified from **Task B**
    - Use `actor.process.pid` as identifier to go across datasources
    - Use `variable.pid` to refer to the value in previously executed Kestrel variable
    - Optional: also use `device.hostname` as an additional identifier in the `WHERE` clause

In [3]:
mde_events = GET event
             FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
             WHERE actor.process.pid = reader.pid
               AND device.hostname = reader.endpoint.hostname

### Task D: Obtain Queries from the MDE Events

**Prerequisite: Task C**

1. Use a `FIND` command to obtain `query_info` associated with the MDE events
2. Display the `query_info` objects

In [4]:
queries = FIND query_info RESPONDED mde_events

DISP queries

uid,attr_list,search_filter
113597,"[""objectClass""]",(objectClass=*)
113598,"[""thumbnailphoto""]",(&(objectclass=contact)(!name=CryptoPolicy)(ThumbnailPhoto=*))
113608,"[""""]",(name=CryptoPolicy)
113616,"[""thumbnailphoto""]",(l=9736f74f-fd37-4b02-80e8-8120a72ad6c2)
113771,"[""""]","(&(objectCategory=user)(memberOf=CN=Domain Admins,CN=Users,DC=simulandlabs,DC=com))"
